# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770837738619                   -0.52    8.0         
  2   -2.772140662512       -2.89       -1.32    1.0    173ms
  3   -2.772170163249       -4.53       -2.52    1.0    132ms
  4   -2.772170649074       -6.31       -3.31    1.0    172ms
  5   -2.772170721904       -7.14       -3.98    2.0    170ms
  6   -2.772170722979       -8.97       -4.84    1.0    138ms
  7   -2.772170723013      -10.46       -5.50    1.0    158ms
  8   -2.772170723015      -11.76       -6.51    2.0    159ms
  9   -2.772170723015      -13.91       -7.35    1.0    149ms
 10   -2.772170723015   +  -14.45       -7.49    2.0    179ms
 11   -2.772170723015      -13.85       -7.81    1.0    151ms
 12   -2.772170723015   +  -14.01       -8.79    1.0    162ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773558058013584

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770798247458                   -0.53    9.0         
  2   -2.772055365593       -2.90       -1.32    1.0    177ms
  3   -2.772083128111       -4.56       -2.55    1.0    177ms
  4   -2.772083387321       -6.59       -3.56    1.0    180ms
  5   -2.772083415804       -7.55       -3.89    2.0    490ms
  6   -2.772083417725       -8.72       -4.83    1.0    141ms
  7   -2.772083417810      -10.07       -5.54    2.0    159ms
  8   -2.772083417811      -12.11       -5.94    2.0    177ms
  9   -2.772083417811      -13.54       -6.68    1.0    157ms
 10   -2.772083417811   +  -14.18       -7.18    1.0    149ms
 11   -2.772083417811      -13.73       -7.61    2.0    170ms
 12   -2.772083417811   +  -14.51       -7.95    1.0    152ms
 13   -2.772083417811   +  -13.75       -8.60    1.0    153ms

Polarizability via ForwardDiff:       1.7725349657106593
Polarizabilit